# Optimization Methods For Data Science
## Final Project - Part 1: Multi-Layer-Perceptron

Géraldine V. Maurer, Viktoriia Vlasenko

### Import Libraries

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

### Functions

In [4]:
# Gauss Kernel
def gauss_kernel(x1, x2, gamma):
    return np.exp(-gamma * np.linalg.norm(x1-x2)**2)

# Polynomial Kernel
def poly_kernel(x1, x2, gamma, p):
    return (np.dot(x1, x2) + 1)**p

### Import Data

In [6]:
df = pd.read_csv("https://raw.githubusercontent.com/gmaurer08/Optimization-Final-Project/refs/heads/main/GENDER_CLASSIFICATION.csv")
df.head()

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,gt
0,-0.900846,0.102587,-0.397814,0.112796,2.588096,-0.192754,-0.968311,-0.490886,-0.872099,-0.288411,...,2.541431,1.739102,0.166066,4.584869,-0.107031,-0.913990,-0.686416,-0.368085,-0.870545,0
1,-0.838868,0.039976,-0.387101,0.055413,2.066874,-0.226948,-0.947416,-0.472817,-0.855387,-0.207101,...,1.991721,1.259745,0.065058,3.019790,-0.110633,-0.890023,-0.611625,-0.298235,-0.855208,0
2,-0.814961,-0.010184,-0.397147,0.092713,1.897454,-0.269387,-0.945285,-0.449579,-0.849705,-0.151179,...,1.822978,1.105511,0.065353,2.500681,-0.052730,-0.885691,-0.583346,-0.218140,-0.856456,0
3,-0.110470,0.027849,-0.044310,-0.005343,0.177831,-0.232092,-0.562700,-0.400713,-0.552356,0.037349,...,-0.098367,-0.370318,-0.123008,-0.861314,0.106840,-0.483669,-0.224164,0.147321,-0.615051,0
4,-0.626313,-0.091985,-0.373756,-0.005083,1.172486,-0.314868,-0.885046,-0.412587,-0.818729,-0.012022,...,1.030348,0.421886,-0.068029,0.258984,-0.057158,-0.834079,-0.441066,-0.099874,-0.829539,0


### Data Preparation

In [12]:
# Split into x and y
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values

# Standardize the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_scaled = sc.fit_transform(X)

# Split into test and train set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_scaled, Y, test_size = 0.2)

In [ ]:
# ----- SVM Dual with CVXOPT + k-fold CV (Question 2) -----

import numpy as np
from cvxopt import matrix, solvers
from sklearn.model_selection import StratifiedKFold

# Keeping CVXOPT quiet so the notebook output stays clean; solver params remain DEFAULT as per assignment
solvers.options['show_progress'] = False   # Optimization routine: CVXOPT 'qp' (settings: DEFAULT)

# quick note: the dual SVM expects labels in {-1,+1}, while your dataset uses {0,1}
# We map once here for the final train/test training stage; CV does its own mapping inside
y_train_svm = np.where(y_train==0, -1.0, 1.0)
y_test_svm  = np.where(y_test==0, -1.0, 1.0)

# helper kernels (vectorized) to avoid Python loops, which would be slow
def rbf_kernel_matrix(X1, X2, gamma: float):
    # Computes pairwise squared Euclidean distances efficiently using ||a-b||^2=||a||^2+||b||^2-2a^Tb
    X1_sq = np.sum(X1**2, axis=1)[:, None]   # each row's squared norm
    X2_sq = np.sum(X2**2, axis=1)[None, :]   # each row's squared norm (broadcast shape)
    # Capitalized: this constructs the full pairwise squared distance matrix
    sq_dists = X1_sq+X2_sq-2*(X1 @ X2.T)
    # returns K_ij=exp(-gamma*||xi-xj||^2)
    return np.exp(-gamma*sq_dists)

def poly_kernel_matrix(X1, X2, p: int):
    # a classic in SVMs: adds a bias 1.0 then raises to power p
    return (X1 @ X2.T+1.0)**p

def make_kernel_mats(Xtr, Xte, kernel_name: str, **kparams):
    # chooses the right kernel and builds train/train and train/test Gram blocks
    if kernel_name=='rbf':
        K_tr = rbf_kernel_matrix(Xtr, Xtr, gamma=kparams['gamma'])
        K_te = rbf_kernel_matrix(Xtr, Xte, gamma=kparams['gamma'])
    elif kernel_name=='poly':
        K_tr = poly_kernel_matrix(Xtr, Xtr, p=kparams['p'])
        K_te = poly_kernel_matrix(Xtr, Xte, p=kparams['p'])
    else:
        # Little guardrail: fail fast if kernel_name is wrong
        raise ValueError("kernel_name must be 'rbf' or 'poly'")
    # tiny jitter for numerical stability (helps positive definiteness in practice)
    K_tr = K_tr+1e-8*np.eye(K_tr.shape[0])
    return K_tr, K_te

# now we solve the dual QP:
#    maximize  1^T a - 1/2 a^T (Y Y^T .* K) a
#    subject to 0<=a_i<=C and y^T a=0
# cvxopt solves a MIN problem, so we flip signs accordingly
def solve_svm_dual(K_tr: np.ndarray, ytr: np.ndarray, C: float):
    n = ytr.shape[0]  # number of training points in this fold
    Y = ytr.reshape(-1, 1)  # column vector
    P = (Y @ Y.T)*K_tr      # This is (y y^T)⊙K (elementwise via broadcasted product)
    # cvxopt form: min (1/2)a^T P a + q^T a, with G a<=h, A a=b
    P_cvx = matrix(P, tc='d')                      # Positive semidefinite matrix for QP
    q_cvx = matrix(-np.ones(n), tc='d')            # q=-1 -> objective matches the negated dual
    # box constraints 0<=a<=C encoded as two stacks: -I a<=0 and I a<=C
    G = np.vstack([-np.eye(n), np.eye(n)])         # Concatenates the two inequality blocks
    h = np.hstack([np.zeros(n), np.full(n, C)])    # Corresponding right-hand sides
    G_cvx = matrix(G, tc='d')
    h_cvx = matrix(h, tc='d')
    # equality constraint enforces y^T a=0 (feasibility on the margin balance)
    A_cvx = matrix(ytr.reshape(1, -1), tc='d')
    b_cvx = matrix(0.0)
    # important: we keep default tolerances/parameters (as requested)
    sol = solvers.qp(P_cvx, q_cvx, G_cvx, h_cvx, A_cvx, b_cvx)
    alphas = np.array(sol['x']).ravel()            # The dual variables a_i
    return alphas, sol, P                          # returning P helps evaluate the dual objective if needed

def compute_b(alphas: np.ndarray, ytr: np.ndarray, K_tr: np.ndarray, C: float):
    # finds the bias b by averaging across margin SVs (0<a_i<C), fallback to all SVs if none are free
    eps = 1e-6
    sv = (alphas>eps)                               # support vectors (nonzero alphas)
    sv_margin = (alphas>eps) & (alphas<C-eps)       # free SVs inside the box
    idx = np.where(sv_margin)[0]
    if idx.size==0:                                 # sometimes all SVs hit the box; average over them instead
        idx = np.where(sv)[0]
    # b = y_i - sum_j a_j y_j K_ji, averaged over chosen SVs
    b_vals = ytr[idx]-(alphas*ytr) @ K_tr[:, idx]
    return float(np.mean(b_vals)), sv               # returns mean bias and the SV mask for stats

def decision_function(alphas, ytr, K_te, b):
    # computes f(x)=sum_i a_i y_i K(x_i,x)+b for a batch via Gram block K_te
    return (alphas*ytr) @ K_te+b

def accuracy_from_scores(scores, y_true_binary01):
    # small bridge: scores>=0 -> class 1 else 0 (consistent with our {-1,+1} mapping)
    y_pred = (scores>=0).astype(int)
    return float(np.mean(y_pred==y_true_binary01))

# cross-validation utilities
from itertools import product

def grid_from_dict(d):
    # expands a dict of lists into a cartesian product of param dicts
    keys = list(d.keys())
    for values in product(*[d[k] for k in keys]):   # note: star here is argument unpacking, not multiplication
        yield dict(zip(keys, values))

def cross_validate_svm(X, y_binary01, kernel_name: str, param_grid: dict, k: int=5):
    # Stratified splits preserve class balance; we remap labels to {-1,+1} per fold
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    best_params, best_mean = None, -np.inf
    history = []  # we’ll keep (params, mean_acc, fold_accs) to report in the write-up if needed
    for params in grid_from_dict(param_grid):
        # lowercase: collect each fold's validation accuracy
        fold_accs = []
        for tr_idx, va_idx in skf.split(X, y_binary01):
            Xtr, Xva = X[tr_idx], X[va_idx]
            ytr01, yva01 = y_binary01[tr_idx], y_binary01[va_idx]
            ytr = np.where(ytr01==0, -1.0, 1.0)
            # Build Gram matrices for this fold; caching could speed up further if needed
            K_tr, K_va = make_kernel_mats(Xtr, Xva, kernel_name, **params)
            # Solve the dual with CVXOPT
            alphas, sol, P = solve_svm_dual(K_tr, ytr, C=params['C'])
            # estimate bias using margin SVs, then evaluate on validation
            b, _ = compute_b(alphas, ytr, K_tr, C=params['C'])
            scores_va = decision_function(alphas, ytr, K_va, b)
            acc_va = accuracy_from_scores(scores_va, yva01)
            fold_accs.append(acc_va)
        mean_acc = float(np.mean(fold_accs))
        history.append((params.copy(), mean_acc, fold_accs.copy()))
        # Capitalized: track the best hyperparameters by mean validation accuracy
        if mean_acc>best_mean:
            best_mean, best_params = mean_acc, params.copy()
    return best_params, best_mean, history

# choose kernel & hyperparameter grid; this is a sensible baseline
KERNEL = 'rbf'  # 'rbf' or 'poly'

if KERNEL=='rbf':
    # gamma sweeps a few orders; C explores under/over-regularization
    param_grid = {
        'C': [0.1, 1, 10, 100],
        'gamma': [0.01, 0.1, 1.0]
    }
elif KERNEL=='poly':
    # polynomial degree p controls model capacity; often p=2..4 is plenty
    param_grid = {
        'C': [0.1, 1, 10, 100],
        'p': [2, 3, 4]
    }
else:
    # friendly failure if someone mistypes the kernel name
    raise ValueError("KERNEL must be 'rbf' or 'poly'")

# run cross-validation to pick (C, kernel hyperparam)
best_params, val_best_mean, cv_history = cross_validate_svm(
    x_train, (y_train>0).astype(int), KERNEL, param_grid, k=5
)

print(f"[CV] Kernel = {KERNEL} | Best params = {best_params} | Mean 5-fold val acc = {val_best_mean:.4f}")

# Train FINAL model on the whole training split using the best hyperparameters from CV
K_tr, K_te = make_kernel_mats(x_train, x_test, KERNEL, **best_params)
alphas, sol, P = solve_svm_dual(K_tr, y_train_svm, C=best_params['C'])
b, sv_mask = compute_b(alphas, y_train_svm, K_tr, C=best_params['C'])

# compute accuracies to report (train/test)
scores_tr = decision_function(alphas, y_train_svm, K_tr, b)
scores_te = decision_function(alphas, y_train_svm, K_te, b)
train_acc = accuracy_from_scores(scores_tr, (y_train>0).astype(int))
test_acc  = accuracy_from_scores(scores_te,  (y_test>0).astype(int))

# optimization diagnostics for the report:
# cvxopt solves min g(a)=1/2 a^T P a - 1^T a ; the dual objective is f(a)=-g(a)
a = alphas
primal_obj_g = float(sol['primal objective'])    # value minimized by cvxopt
final_dual_obj = -primal_obj_g                   # flip sign to get maximized dual
initial_dual_obj = 0.0                           # at a=0 the dual is 0
iterations = int(sol['iterations'])
num_sv = int(np.sum(sv_mask))

print("\n--- RESULTS (Question 2) ---")
print(f"Kernel: {KERNEL}")
print(f"Best hyperparameters (via 5-fold CV): {best_params}")
print(f"Validation (mean over folds): {val_best_mean:.4f}")
print(f"Train accuracy: {train_acc:.4f}")
print(f"Test  accuracy: {test_acc:.4f}")
print(f"Support vectors: {num_sv} / {len(y_train_svm)}")

print("\nOptimization performance (final training run):")
print(f"  Optimization routine: CVXOPT 'qp' (parameters: DEFAULT)")
print(f"  Dual objective: initial = {initial_dual_obj:.6f}, final = {final_dual_obj:.6f}")
print(f"  Iterations: {iterations}")
